#Proyecto final Machine Learning
##Bot de preguntas y respuestas

### Integrantes
* José Estensoro (josee906@gmail.com)
* Roger Patón (oviroger@gmail.com)


## Descripcion del Problema 

El objetivo es crear un chatbot que pueda responder preguntas en ingles sobre temas de cultura general. Para lograr esto, se utilizará el modelado de temas en los datos de preguntas frecuentes y respuestas proporcionados un. dataset conseguido en Kaggle.

Usando técnicas de Machine learning, podemos identificar los temas principales en los datos y categorizar las preguntas y respuestas varios tipos de temas:


Una vez que hemos categorizado las preguntas y respuestas en los temas, podemos entrenar un modelo, se tendra que elegir un modelo entre todos los aprendidos en la materia, para poder hacer el entrenamiento del bot para que bot pueda responder preguntas.





##Instalamos la librerias necesarias

In [1]:
#Para realizar el proyecto va a ser necesario utilizar la libreria tflearn
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=e5058236015d48490ca576c55f62c93f71a409f21d0965d88c465d489b043258
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [11]:
#Para poder realizar el analisis del modelo se utilizara tensorboard
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importamos la librerias a usar


In [2]:
#NLTK 
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize
#Numpy
from numpy.core.defchararray import mod
from numpy.lib.function_base import append
import numpy
#tflearn
import tflearn
import pickle,json, pandas as pd
import matplotlib.pyplot as plt
stemmer = LancasterStemmer()

Instructions for updating:
non-resource variables are not supported in the long term


#Cargado del dataset y preparacion de los datos 
El dataset es importando desde una carpeta del Google Drive

In [3]:
df = pd.read_csv('/content/drive/MyDrive/MachineL/proyfinal/WikiQA.tsv',sep='\t')
df.head()

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-0,African immigration to the United States refer...,0
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-1,The term African in the scope of this article ...,0
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-2,From the Immigration and Nationality Act of 19...,0
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-3,African immigrants in the United States come f...,0
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,D0-4,"They include people from different national, l...",0


Se agrupan. todas las columnas que tengan la misma oracion en la columna "Question".

In [4]:
df = df.groupby(["QuestionID","Question","DocumentID","DocumentTitle"])['Sentence'].apply(', '.join).reset_index()
df

,QuestionID,Question,DocumentID,DocumentTitle,Sentence
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0,African immigration to the United States,African immigration to the United States refer...
1,Q1,how are glacier caves formed?,D1,Glacier cave,A partly submerged glacier cave on Perito More...
2,Q10,how an outdoor wood boiler works,D10,Outdoor wood-fired boiler,The outdoor wood boiler is a variant of the cl...
3,Q100,what happens to the light independent reactio...,D100,Light-independent reactions,The simplified internal structure of a chlorop...
4,Q1000,where in the bible that palestine have no land...,D965,Philistines,"The Philistine cities of Gaza, Ashdod, Ashkelo..."
...,...,...,...,...,...
3039,Q995,what does it mean for a ship to be flagged by...,D960,Maritime flag,A medieval ship flag captured by forces from L...
3040,Q996,what kind of earthquake was haiti,D961,2010 Haiti earthquake,The 2010 Haiti earthquake was a catastrophic m...
3041,Q997,what kind of horse was mr ed,D962,Mister Ed,"Mister Ed is a fictional talking horse , origi..."
3042,Q998,What does Rapture meaning in a theological sense?,D963,Rapture,The Rapture is a term in Christian eschatology...


se descraga la libreria punkt de NLTK

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Se realiza el preprosesamiento de los datos, se los tokeniza y se aplica Steamming 

In [6]:
words = []
labels = [] 
docs_x = []
docs_y = []


for index, row in df.iterrows():
      wrds = nltk.word_tokenize(row["Question"])
      words.extend(wrds)
      docs_x.append(wrds)
      docs_y.append(row["DocumentTitle"])
      if row["DocumentTitle"] not in labels:
          labels.append(row["DocumentTitle"])
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))

labels =  sorted(labels)

training  = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    
    output_row =  out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)

#Se crea el modelo con una red neuronal profunda
Se crea un modelo de 5 capaz con 100 nodos cada uno para


In [7]:
net = tflearn.input_data(shape=[None, len(training[0])])

net =  tflearn.fully_connected(net , 100)
net =  tflearn.fully_connected(net , 100)
net =  tflearn.fully_connected(net , 100)
net =  tflearn.fully_connected(net , 100)
net =  tflearn.fully_connected(net , 100)


net =  tflearn.fully_connected(net , len(output[0]), activation="softmax")
net =  tflearn.regression(net)

model = tflearn.DNN(net,tensorboard_verbose=3)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Entrenamos el modelo

In [13]:
model.fit(training, output, n_epoch=90, batch_size=32, show_metric=True)


Training Step: 8639  | total loss: 0.71970 | time: 12.678s
| Adam | epoch: 090 | loss: 0.71970 - acc: 0.8251 -- iter: 3040/3044
Training Step: 8640  | total loss: 0.71715 | time: 12.791s
| Adam | epoch: 090 | loss: 0.71715 - acc: 0.8301 -- iter: 3044/3044
--


si es que se entreno el modelo recientemente se lo guarda

In [27]:
model.save("/content/drive/MyDrive/MachineL/proyfinal/model/model.tflearn") 

Si ya se tiene un modelo existente se lo sube al colab, se lo descomprime y se lo carga al modelo

In [28]:
model.load("/content/drive/MyDrive/MachineL/proyfinal/model/model.tflearn")

#Visualizamos el modelo


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/tflearn_logs

Se utiliza el metodo bag of word para las respuestas

In [17]:
def bag_of_words(s, words = words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i , w in enumerate(words):
            if w == se:
                bag[i]= 1
    return numpy.array(bag)

#Probamos el modelo

se obtiene la respuesta segun la pregunta 

In [18]:
#para que la respuesta salga completa
pd.options.display.max_colwidth = None

#Primera pregunta

palabra = "where does limestone form"

result = model.predict([bag_of_words(palabra, words)])[0]
result_index = numpy.argmax(result)
tag = labels[result_index]
result = df[df['DocumentTitle'] == tag]
print(result["Sentence"])



#Segunda pregunta

palabra = "AFRICAN AMERICANS WERE IMMIGRATED US "

result = model.predict([bag_of_words(palabra, words)])[0]
result_index = numpy.argmax(result)
tag = labels[result_index]
result = df[df['DocumentTitle'] == tag]
print(result["Sentence"])
#Tercera pregunta
palabra = "earthquake was haiti"

result = model.predict([bag_of_words(palabra, words)])[0]
result_index = numpy.argmax(result)
tag = labels[result_index]
result = df[df['DocumentTitle'] == tag]
print(result["Sentence"])
#Cuarta pregunata      
palabra = "What does Rapture meaning in a theological sense?"

result = model.predict([bag_of_words(palabra, words)])[0]
result_index = numpy.argmax(result)
tag = labels[result_index]
result = df[df['DocumentTitle'] == tag]
print(result["Sentence"])


60    Limestone is a sedimentary rock composed largely of the minerals calcite and aragonite , which are different crystal forms of calcium carbonate (CaCO3)., Many limestones are composed from skeletal fragments of marine organisms such as coral or foraminifera ., Limestone makes up about 10% of the total volume of all sedimentary rocks., The solubility of limestone in water and weak acid solutions leads to karst landscapes, in which water erodes the limestone over thousands to millions of years., Most cave systems are through limestone bedrock., Limestone has numerous uses: as a building material , as aggregate for the base of roads, as white pigment or filler in products such as toothpaste or paints, and as a chemical feedstock ., The first geologist to distinguish limestone from dolomite was Belsazar Hacquet in 1778.
Name: Sentence, dtype: object
1607    Where Do We Come From?, What Are We?, Where Are We Going?, One of Post-Impressionist Paul Gauguin 's most famous paintings., The 